# New Nelder Mead?

Now using a penalty function, can we get scipy's nelder mead function to work?

Quadratic penalty function: 

$$f(x) = g(x) + \lambda (d(x))^2$$

$d(x)$ is now the distance away from water


## Setup Imports

(not relevant to repo, can skip) 

Adding module path to run correctly in examples folder

In [4]:
import sys
import os

# Add the path to the folder containing the module
module_path = os.path.abspath(os.path.join('..'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

/Users/gracekim/Documents/School_Everything_and_LEARNING/Stanford/Githubs/loc-gsopt/src


In [5]:
# !{sys.executable} -m pip install geopy

# !{sys.executable} -m pip install global-land-mask

## Imports

In [6]:
from common.sat_gen import satellites_from_constellation
from common.station_gen import gs_json, return_bdm_gs
from common.utils import load_earth_data, compute_all_gaps_contacts, compute_earth_interior_angle
# from optimizations.nelder_mead import nelder_mead

import numpy as np

# Brahe Imports
import brahe as bh
import brahe.data_models as bdm
import brahe.access.access as ba

# random points function
import random_land_points as rlp

# Plotting Imports
import shapely
import cartopy.crs as ccrs
import cartopy.geodesic
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML # disply inline for ipynb 

## Setup for Earth Inertial Data and Scenario Generation

In [ ]:
############################### SETUP: ###############################

# Make sure to load in earth inertial data every start time!
load_earth_data('data/iau2000A_finals_ab.txt')

############################### STEP 1: Satellites ###############################

# Loading in a satellite constellation 
CONSTELLATIONS = sorted(['YAM', 'UMBRA', 'SKYSAT', 'ICEYE', 'FLOCK', 'HAWK', 'CAPELLA', 'LEGION', 'WORLDVIEW', 'GEOEYE',
                  'NUSAT'])

constellation = CONSTELLATIONS[3]

satellites = satellites_from_constellation(constellation)

############################### STEP 2: Scenario Generation ###############################

# Setting up Epochs
epc0 = bh.Epoch(2024, 5, 20, 0, 0, 0) # This is the epoch of the orbital elements
epc10 = epc0 + 86400 # make this longer time period

continent = 'Antarctica' # 'North America'  


Loading the latest Earth Orientation Data


## Nelder Mead - Scipy

,geometry
0,"MULTIPOLYGON (((180 -16.06713, 180 -16.55522, ..."
1,"POLYGON ((33.90371 -0.95, 34.07262 -1.05982, 3..."
2,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,"MULTIPOLYGON (((-122.84 49, -122.97421 49.0025..."
4,"MULTIPOLYGON (((-122.84 49, -120 49, -117.0312..."
...,...
172,"POLYGON ((18.82982 45.90887, 18.82984 45.90888..."
173,"POLYGON ((20.0707 42.58863, 19.80161 42.50009,..."
174,"POLYGON ((20.59025 41.85541, 20.52295 42.21787..."
175,"POLYGON ((-61.68 10.76, -61.105 10.89, -60.895..."


In [9]:
import geopandas as gpd
from shapely.geometry import Point
from geopy.distance import geodesic


# Load land boundary dataset (replace with your actual file)
land_data = gpd.read_file("data/ne_10m_admin_0_countries.shp")  # Use a Natural Earth shapefile

# Define your latitude and longitude point in the ocean
ocean_point = Point(-30.0, -60.0)  # Example: (lat, longitude)


def calculate_distance_to_land(point, land_geometries):
    # Initialize minimum distance and closest land point
    min_distance = float('inf')
    closest_land_point = None

    for land_poly in land_geometries:
        # Check if the point is on land
        if land_poly.contains(point):
            return 0, point  # The point is on land
        
        # Handle MultiPolygon or Polygon
        if land_poly.geom_type == "MultiPolygon":
            for poly in land_poly.geoms:  # Iterate over individual polygons
                nearest_point = poly.exterior.interpolate(
                    poly.exterior.project(point)
                )
                dist = geodesic((point.y, point.x), (nearest_point.y, nearest_point.x)).meters
                if dist < min_distance:
                    min_distance = dist
                    closest_land_point = nearest_point
        elif land_poly.geom_type == "Polygon":
            nearest_point = land_poly.exterior.interpolate(
                land_poly.exterior.project(point)
            )
            dist = geodesic((point.y, point.x), (nearest_point.y, nearest_point.x)).meters
            if dist < min_distance:
                min_distance = dist
                closest_land_point = nearest_point

    return min_distance, closest_land_point
# Extract geometries of the land boundaries
land_geometries = land_data['geometry']

# Calculate distance to the nearest land
distance, closest_land_point = calculate_distance_to_land(ocean_point, land_geometries)


In [63]:
distance

159044.35435175823

In [10]:
import numpy as np
from scipy.optimize import minimize

from global_land_mask import globe

global_list_of_simplexes = []

# Load land boundary dataset (replace with your actual file)
land_data = gpd.read_file("data/ne_10m_admin_0_countries.shp")  # Use a Natural Earth shapefile

def penalty(new_gs):
    on_water = False
    on_water = globe.is_ocean(new_gs[1], new_gs[0])
    if on_water:
        # Calculate distance to the nearest lan
        distance, closest_land_point = calculate_distance_to_land(Point(new_gs[0], new_gs[1]), land_geometries)
        return distance
    return 0

def cost_func_gap(new_gs, gs_list = [], satellites=satellites[0:10], epc_start = epc0, epc_end = epc10, plot = True):    
    gs_list = [return_bdm_gs(new_gs[0], new_gs[1])]
    global_list_of_simplexes.append([new_gs[0], new_gs[1]])
    _, _, gaps_seconds = compute_all_gaps_contacts(satellites, gs_list ,epc_start, epc_end, plot)

    value = np.mean(gaps_seconds) #penalty(new_gs)
    print(penalty(new_gs)/1000)
    print(value)
    value +=(penalty(new_gs)/1000)**2
    print(gs_list)


    return value

# List to store simplex points
simplex_history = []

def callback(xk, convergence=None):
    simplex_history.append(xk.copy())

# Initial guess for the coordinates (x, y)
initial_guess = np.array([-30, -60])

# Perform the optimization using Nelder-Mead
result = minimize(cost_func_gap, 
                  initial_guess, 
                  method='Nelder-Mead',
                  options={'disp': True,
                          'maxiter': 5,
                           'initial_simplex' : np.array([[-30, -60],[-100, -60],[-30, -90]])},
                  callback=callback,
                  bounds = ((-180,180),(-90,90)))

/Users/gracekim/Documents/School_Everything_and_LEARNING/Stanford/Githubs/loc-gsopt/src/common/utils.py:77: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()
/Users/gracekim/Documents/School_Everything_and_LEARNING/Stanford/Githubs/loc-gsopt/src/common/utils.py:78: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


159.04435435175822
7226.248136330357
[Station(type='Feature', geometry=GeoJSONGeometry(type='Point', coordinates=[-30.0, -60.0]), properties=StationProperties(id=UUID('93705748-f32f-4b75-99f0-a273d40d4529'), station_name='', constraints=AccessConstraints(ascdsc=<AscendingDescending.either: 'either'>, look_direction=<LookDirection.either: 'either'>, look_angle_min=0.0, look_angle_max=90.0, elevation_min=0.0, elevation_max=90.0), downlink_rate_max=0.0))]
1070.8624291832396
7237.187069517858
[Station(type='Feature', geometry=GeoJSONGeometry(type='Point', coordinates=[-100.0, -60.0]), properties=StationProperties(id=UUID('3c058663-378c-4020-8527-81b9d941fbe6'), station_name='', constraints=AccessConstraints(ascdsc=<AscendingDescending.either: 'either'>, look_direction=<LookDirection.either: 'either'>, look_angle_min=0.0, look_angle_max=90.0, elevation_min=0.0, elevation_max=90.0), downlink_rate_max=0.0))]
0.0
4718.554312173914
[Station(type='Feature', geometry=GeoJSONGeometry(type='Point',

/var/folders/d9/89_mskkx2p1bqvlhj4jgcc8h0000gn/T/ipykernel_56089/2800337223.py:44: RuntimeWarning: Maximum number of iterations has been exceeded.
  result = minimize(cost_func_gap,


In [11]:
result

       message: Maximum number of iterations has been exceeded.
       success: False
        status: 2
           fun: 4718.554312173914
             x: [-3.000e+01 -9.000e+01]
           nit: 5
          nfev: 14
 final_simplex: (array([[-3.000e+01, -9.000e+01],
                       [-1.250e+01, -9.000e+01],
                       [-1.688e+01, -9.000e+01]]), array([ 4.719e+03,  4.719e+03,  4.719e+03]))

In [12]:

# Display results
print("Optimization Result:", result)
print("Simplex History:")
for i, simplex in enumerate(simplex_history):
    print(f"Iteration {i+1}: {simplex}")

Optimization Result:        message: Maximum number of iterations has been exceeded.
       success: False
        status: 2
           fun: 4718.554312173914
             x: [-3.000e+01 -9.000e+01]
           nit: 5
          nfev: 14
 final_simplex: (array([[-3.000e+01, -9.000e+01],
                       [-1.250e+01, -9.000e+01],
                       [-1.688e+01, -9.000e+01]]), array([ 4.719e+03,  4.719e+03,  4.719e+03]))
Simplex History:
Iteration 1: [-30. -90.]
Iteration 2: [-30. -90.]
Iteration 3: [-30. -90.]
Iteration 4: [-30. -90.]


In [13]:

points = []

# random locations
for i in range(3):
    point = rlp.random_points(continent)[0] # Get a random point on land in Antartica
    points.append((point[0],point[1]))

# Just put the random points here for now, maybe we do this for every continent?
# points = [[ 179.4515625, 51.37260742],
#  [ -61.17426758, 10.07802734],
#  [-178.19453125, 51.88222656]]


# Cost function, we should be able tochange this if needed?
def cost_func_gap(new_gs, gs_list = [], satellites=satellites[0:10], epc_start = epc0, epc_end = epc10, plot = False):    
    gs_list.append(return_bdm_gs(new_gs[0], new_gs[1]))
    _, _, gaps_seconds = compute_all_gaps_contacts(satellites, gs_list ,epc_start, epc_end, plot)
    return np.mean(gaps_seconds)

print(points)
plot_points = nelder_mead(points, cost_func_gap, 5, continent)
print(plot_points)

[(-120.50620928728756, -78.02178832357369), (-130.7170688584358, -89.3838462865147), (-136.01605192508396, -75.67401146864609)]


NameError: name 'nelder_mead' is not defined

In [14]:



############################### STEP 4: Plotting ###############################

# Now we can plot what a satellite can see from a given altitude
# Let's say we want to see what a satellite at 525 km can see if all observers
# are looking at it with at least 20 degrees elevation angle. This is an appropriate
# Value for a communications (Starlink/Kuiper) user terminal. If you wanted to get
# the maximum possible coverage limited by the Earth's curvature, you would use 0 degrees.

alt = 570 # Altitude in km
elevation_min = 20.0
lam = compute_earth_interior_angle(ele=elevation_min, alt=alt)

def sliding_window(data, window_size=3):
    """
    Creates a list of lists by applying a sliding window over the input list.

    Args:
        data (list): The input list to process.
        window_size (int): The size of the window (default is 3).

    Returns:
        list: A list of sublists, each containing `window_size` elements.
    """
    if window_size > len(data):
        raise ValueError("Window size cannot be larger than the data length.")

    result = [data[i:i + window_size] for i in range(len(data) - window_size + 1)]
    return result

def animate(i):
    fig.clear()
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_global()
    ax.stock_img()
    points = plot_points[i]
    print(i)
    print(points)

    ax.set_title("Animation Counter Optimization: "+ str(i))
    ax.scatter(*zip(*points))
    for point in points:
        # Get a bunch of points in a circle space at the the right angle offset from the sub-satellite point to draw the view cone
        circle_points = cartopy.geodesic.Geodesic().circle(lon=point[0], lat=point[1], radius=lam*bh.R_EARTH, n_samples=100, endpoint=False)
        geom = shapely.geometry.Polygon(circle_points)
        ax.add_geometries((geom,), crs=ccrs.Geodetic(), alpha=0.5, edgecolor='none', linewidth=0)

    ax.set_yticks(np.arange(-90, 90, 30))
    ax.set_xticks(np.arange(-180, 180, 30))
    # sc = ax.scatter(x=longs, y=lats, c=gaps,cmap = 'cool')
    # plt.colorbar(sc)
    plt.grid()
    # plt.show()

plot_points = sliding_window(global_list_of_simplexes)
print(plot_points)

fig = plt.figure()
ani = FuncAnimation(fig, animate, frames = range(len(plot_points)),repeat = False,interval =500)

HTML(ani.to_html5_video())
# plt.ylim(-1000,1000)
# plt.xlim(-1000,1000)
# plt.show()
# ani.save(filename="example.gif", writer="ffmpeg")


[[[-30.0, -60.0], [-100.0, -60.0], [-30.0, -90.0]], [[-100.0, -60.0], [-30.0, -90.0], [40.0, -90.0]], [[-30.0, -90.0], [40.0, -90.0], [40.0, -90.0]], [[40.0, -90.0], [40.0, -90.0], [22.5, -90.0]], [[40.0, -90.0], [22.5, -90.0], [-12.5, -90.0]], [[22.5, -90.0], [-12.5, -90.0], [13.75, -90.0]], [[-12.5, -90.0], [13.75, -90.0], [5.0, -90.0]], [[13.75, -90.0], [5.0, -90.0], [-3.75, -90.0]], [[5.0, -90.0], [-3.75, -90.0], [-21.25, -90.0]], [[-3.75, -90.0], [-21.25, -90.0], [-8.125, -90.0]], [[-21.25, -90.0], [-8.125, -90.0], [-12.5, -90.0]], [[-8.125, -90.0], [-12.5, -90.0], [-16.875, -90.0]]]
0
[[-30.0, -60.0], [-100.0, -60.0], [-30.0, -90.0]]
0
[[-30.0, -60.0], [-100.0, -60.0], [-30.0, -90.0]]


/Users/gracekim/.pyenv/versions/3.11.9/envs/gsopt/lib/python3.11/site-packages/cartopy/mpl/geoaxes.py:790: UserWarning: Approximating coordinate system <Geographic 2D CRS: +proj=lonlat +datum=WGS84 +ellps=WGS84 +no_defs +t ...>
Name: unknown
Axis Info [ellipsoidal]:
- lon[east]: Longitude (degree)
- lat[north]: Latitude (degree)
Area of Use:
- undefined
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich
 with the PlateCarree projection.
  warnings.warn(f'Approximating coordinate system {crs!r} with '
/Users/gracekim/.pyenv/versions/3.11.9/envs/gsopt/lib/python3.11/site-packages/cartopy/mpl/geoaxes.py:790: UserWarning: Approximating coordinate system <Geographic 2D CRS: +proj=lonlat +datum=WGS84 +ellps=WGS84 +no_defs +t ...>
Name: unknown
Axis Info [ellipsoidal]:
- lon[east]: Longitude (degree)
- lat[north]: Latitude (degree)
Area of Use:
- undefined
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich
 with the PlateCarree pro

1
[[-100.0, -60.0], [-30.0, -90.0], [40.0, -90.0]]
2
[[-30.0, -90.0], [40.0, -90.0], [40.0, -90.0]]


/Users/gracekim/.pyenv/versions/3.11.9/envs/gsopt/lib/python3.11/site-packages/cartopy/mpl/geoaxes.py:790: UserWarning: Approximating coordinate system <Geographic 2D CRS: +proj=lonlat +datum=WGS84 +ellps=WGS84 +no_defs +t ...>
Name: unknown
Axis Info [ellipsoidal]:
- lon[east]: Longitude (degree)
- lat[north]: Latitude (degree)
Area of Use:
- undefined
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich
 with the PlateCarree projection.
  warnings.warn(f'Approximating coordinate system {crs!r} with '
/Users/gracekim/.pyenv/versions/3.11.9/envs/gsopt/lib/python3.11/site-packages/cartopy/mpl/geoaxes.py:790: UserWarning: Approximating coordinate system <Geographic 2D CRS: +proj=lonlat +datum=WGS84 +ellps=WGS84 +no_defs +t ...>
Name: unknown
Axis Info [ellipsoidal]:
- lon[east]: Longitude (degree)
- lat[north]: Latitude (degree)
Area of Use:
- undefined
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich
 with the PlateCarree pro

3
[[40.0, -90.0], [40.0, -90.0], [22.5, -90.0]]
4
[[40.0, -90.0], [22.5, -90.0], [-12.5, -90.0]]
5
[[22.5, -90.0], [-12.5, -90.0], [13.75, -90.0]]


/Users/gracekim/.pyenv/versions/3.11.9/envs/gsopt/lib/python3.11/site-packages/cartopy/mpl/geoaxes.py:790: UserWarning: Approximating coordinate system <Geographic 2D CRS: +proj=lonlat +datum=WGS84 +ellps=WGS84 +no_defs +t ...>
Name: unknown
Axis Info [ellipsoidal]:
- lon[east]: Longitude (degree)
- lat[north]: Latitude (degree)
Area of Use:
- undefined
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich
 with the PlateCarree projection.
  warnings.warn(f'Approximating coordinate system {crs!r} with '
/Users/gracekim/.pyenv/versions/3.11.9/envs/gsopt/lib/python3.11/site-packages/cartopy/mpl/geoaxes.py:790: UserWarning: Approximating coordinate system <Geographic 2D CRS: +proj=lonlat +datum=WGS84 +ellps=WGS84 +no_defs +t ...>
Name: unknown
Axis Info [ellipsoidal]:
- lon[east]: Longitude (degree)
- lat[north]: Latitude (degree)
Area of Use:
- undefined
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich
 with the PlateCarree pro

6
[[-12.5, -90.0], [13.75, -90.0], [5.0, -90.0]]
7
[[13.75, -90.0], [5.0, -90.0], [-3.75, -90.0]]
8
[[5.0, -90.0], [-3.75, -90.0], [-21.25, -90.0]]


/Users/gracekim/.pyenv/versions/3.11.9/envs/gsopt/lib/python3.11/site-packages/cartopy/mpl/geoaxes.py:790: UserWarning: Approximating coordinate system <Geographic 2D CRS: +proj=lonlat +datum=WGS84 +ellps=WGS84 +no_defs +t ...>
Name: unknown
Axis Info [ellipsoidal]:
- lon[east]: Longitude (degree)
- lat[north]: Latitude (degree)
Area of Use:
- undefined
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich
 with the PlateCarree projection.
  warnings.warn(f'Approximating coordinate system {crs!r} with '
/Users/gracekim/.pyenv/versions/3.11.9/envs/gsopt/lib/python3.11/site-packages/cartopy/mpl/geoaxes.py:790: UserWarning: Approximating coordinate system <Geographic 2D CRS: +proj=lonlat +datum=WGS84 +ellps=WGS84 +no_defs +t ...>
Name: unknown
Axis Info [ellipsoidal]:
- lon[east]: Longitude (degree)
- lat[north]: Latitude (degree)
Area of Use:
- undefined
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich
 with the PlateCarree pro

9
[[-3.75, -90.0], [-21.25, -90.0], [-8.125, -90.0]]
10
[[-21.25, -90.0], [-8.125, -90.0], [-12.5, -90.0]]
11
[[-8.125, -90.0], [-12.5, -90.0], [-16.875, -90.0]]


/Users/gracekim/.pyenv/versions/3.11.9/envs/gsopt/lib/python3.11/site-packages/cartopy/mpl/geoaxes.py:790: UserWarning: Approximating coordinate system <Geographic 2D CRS: +proj=lonlat +datum=WGS84 +ellps=WGS84 +no_defs +t ...>
Name: unknown
Axis Info [ellipsoidal]:
- lon[east]: Longitude (degree)
- lat[north]: Latitude (degree)
Area of Use:
- undefined
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich
 with the PlateCarree projection.
  warnings.warn(f'Approximating coordinate system {crs!r} with '
/Users/gracekim/.pyenv/versions/3.11.9/envs/gsopt/lib/python3.11/site-packages/cartopy/mpl/geoaxes.py:790: UserWarning: Approximating coordinate system <Geographic 2D CRS: +proj=lonlat +datum=WGS84 +ellps=WGS84 +no_defs +t ...>
Name: unknown
Axis Info [ellipsoidal]:
- lon[east]: Longitude (degree)
- lat[north]: Latitude (degree)
Area of Use:
- undefined
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich
 with the PlateCarree pro